<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watson OpenScale and Batch Processing:<br>Apache Spark on Cloud Pak for Data with IBM Analytics Engine

This notebook must be run in the Python 3.10 runtime environment. It requires Watson OpenScale service credentials.

The notebook configures Watson OpenScale to monitor the German Credit Risk model. Use the notebook to enable quality, drift, drift v2, fairness and explainability monitoring and run on-demand evaluations. Before you can run the notebook, you must have the following resources:

1. The common configuration JSON, Drift, Drift v2 Configuration and Explainability Configuration archives generated by using the [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb).
2. Feedback, payload, drifted transactions, explanations queue and result tables in an Apache Hive data warehouse that use the data description language statements (DDLs) that are generated as part of running the previous common configuration notebook.

## Contents

* [1. Setup](#setup)
* [2. Configure Watson OpenScale](#openscale)
* [3. Set up Subscription](#subscription)
* [4. Quality monitoring](#quality)
* [5. Drift monitoring](#drift)
* [6. Drift v2 monitoring](#drift_v2)
* [6. Fairness monitoring](#fairness)
* [7. Explainability monitoring](#explainability)

# 1. Setup <a name="setup"></a>

## Package installation

First import some of the packages you need to use. After you finish installing the following software packages, restart the kernel.

In [1]:
import warnings
warnings.filterwarnings('ignore')
%env PIP_DISABLE_PIP_VERSION_CHECK=1

env: PIP_DISABLE_PIP_VERSION_CHECK=1


In [ ]:
!pip install -U ibm-watson-openscale --no-cache | tail -n 1

Note: you may need to restart the kernel to use updated packages.


## Configure credentials

Provide your IBM Watson OpenScale credentials in the following cell:

In [4]:
WOS_CREDENTIALS = {
    "url": "",
    "username": "",
    "password": "",
    "instance_id": ""
}

## Specify model details

### Service provider and subscription metadata

In [5]:
# Service Provider

SERVICE_PROVIDER_NAME = ""
SERVICE_PROVIDER_DESCRIPTION = ""

# Subscription

SUBSCRIPTION_NAME = ""
SUBSCRIPTION_DESCRIPTION = ""

### IBM Analytics Engine - Spark

Make sure that the Apache Spark manager on IBM Analytics Engine is running, and then provide the following details:

- IAE_SPARK_DISPLAY_NAME: _Display Name of the Spark instance in IBM Analytics Engine_
- IAE_SPARK_JOBS_ENDPOINT: _Spark Jobs Endpoint for IBM Analytics Engine_
- IBM_CPD_VOLUME: _IBM Cloud Pak for Data storage volume name_
- IBM_CPD_USERNAME: _IBM Cloud Pak for Data username_
- IBM_CPD_APIKEY: _IBM Cloud Pak for Data API key_
- IAE_SPARK_DESCRIPTION: _Custom description for the Spark instance_

In [6]:
IAE_SPARK_DISPLAY_NAME = ""
IAE_SPARK_JOBS_ENDPOINT = ""
IBM_CPD_VOLUME = ""
IBM_CPD_USERNAME = ""
IBM_CPD_APIKEY = ""
IAE_SPARK_NAME = ""
IAE_SPARK_DESCRIPTION = ""

#### Provide Spark Resource Settings

To configure how much of your Spark Cluster resources this job can consume, edit the following values:

- max_num_executors: _Maximum Number of executors to launch for this session_
- min_executors: _Minimum Number of executors to launch for this session_
- executor_cores: _Number of cores to use for each executor_   
- executor_memory: _Amount of memory (in GBs) to use per executor process_
- driver_cores: _Number of cores to use for the driver process_
- driver_memory: _Amount of memory (in GBs) to use for the driver process_

In [7]:
spark_parameters = {
    "max_num_executors": 2,
    "min_num_executors": 1,
    "executor_cores": 3,
    "executor_memory": 2,
    "driver_cores": 2,
    "driver_memory": 2
}

### Apache Hive

To connect to Apache Hive, you must provide the following details:

- HIVE_CONNECTION_NAME: _Custom display name for the Hive Connection_
- HIVE_CONNECTION_DESCRIPTION: _Custom description for the Hive connection_
- HIVE_METASTORE_URI: _Thrift URI for Hive Metastore to connect to_

In case of a kerberos-enabled hive, provide additional details required to obtain the Hadoop delegation token:
- HIVE_KERBEROS_PRINCIPAL: The kerberos principal used to generate the delegation token
- DELEGATION_TOKEN_SECRET_URN: The secret_urn of the CP4D vault where the token is stored *OR*
- DELEGATION_TOKEN_ENDPOINT: The REST endpoint which generates and returns the delegation token

In [8]:
HIVE_CONNECTION_NAME = ""
HIVE_CONNECTION_DESCRIPTION = ""
HIVE_METASTORE_URI = ""

# Flag to indicate if the Hive is secured with kerberos
KERBEROS_ENABLED = False
# Provide Hadoop delegation token details if KERBEROS_ENABLED is True
# Provide either secret_urn of the CP4D vault OR the delegation token endpoint. One of the two fields is mandatory to fetch the delegation token.
HIVE_KERBEROS_PRINCIPAL = ""
DELEGATION_TOKEN_SECRET_URN = ""
DELEGATION_TOKEN_ENDPOINT = ""

### Feedback table metadata

The quality monitor stores metadata in the feedback table. To configure the quality monitor, you must provide the following details. To skip quality monitoring, run the following cell to initialize variables with the value of `None`.

- FEEDBACK_DATABASE_NAME: _Database name where feedback table is present_
- FEEDBACK_SCHEMA_NAME: _Schema name where feedback table is present_
- FEEDBACK_TABLE_NAME: _Name of the feedback table_

In [9]:
#feedback

FEEDBACK_DATABASE_NAME = ""
FEEDBACK_TABLE_NAME = ""

### Payload and drift table metadata

The drift monitor stores metadata in the payload and drift tables. To configure the drift monitor, you must provide the following details. To skip drift monitoring, run the following cell to initialize variables with the value of `None`.

- PAYLOAD_DATABASE_NAME: _Database name where payload logging table is present_
- PAYLOAD_SCHEMA_NAME: _Schema name where payload logging table is present_
- PAYLOAD_TABLE_NAME: _Name of the payload logging table_
- DRIFT_DATABASE_NAME: _Database name where drifted transactions table is present_
- DRIFT_SCHEMA_NAME: _Schema name where drifted transactions table is present_
- DRIFT_TABLE_NAME: _Name of the drifted transactions table_

In [10]:
#payload logging

PAYLOAD_DATABASE_NAME = None
PAYLOAD_TABLE_NAME = None

#drift

DRIFT_DATABASE_NAME = None
DRIFT_TABLE_NAME = None

### Explainability table metadata

The explainability monitor requires the queue and result tables. The payload table can also be used as the queue table. To configure the explainability monitor, you must provide the following details. To skip explainability monitoring, run the following cell to initialize variables with the value of `None`.

- EXPLAINABILITY_DATABASE_NAME: _Database name where explanations queue, result tables are present_
- EXPLAINABILITY_QUEUE_TABLE_NAME: _Name of the explanations queue table_
- EXPLAINABILITY_RESULT_TABLE_NAME: _Name of the explanations result table_

In [11]:
#explainability

EXPLAINABILITY_DATABASE_NAME = None
EXPLAINABILITY_QUEUE_TABLE_NAME = None
EXPLAINABILITY_RESULT_TABLE_NAME = None

# 2. Configure Watson OpenScale <a name="openscale"></a>

### Import the required libraries and set up the Watson OpenScale client

In [12]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *

authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS["url"],
        username=WOS_CREDENTIALS["username"],
        password=WOS_CREDENTIALS["password"],
        disable_ssl_verification=True
    )

wos_client = APIClient(authenticator=authenticator, service_url=WOS_CREDENTIALS["url"], service_instance_id=WOS_CREDENTIALS["instance_id"])
data_mart_id=WOS_CREDENTIALS["instance_id"]

### Display Watson OpenScale datamart details

In [13]:
wos_client.data_marts.show()

AIOSFASTPATHICP-00000000-0000-0000-0000-000000000000,Data Mart created by OpenScale ExpressPath,False,active,2024-11-28 19:42:06.911000+00:00,00000000-0000-0000-0000-000000000000


### Create a service provider

In [14]:
# Delete existing service provider with the same name as provided

service_providers = wos_client.service_providers.list().result.service_providers
for provider in service_providers:
    if provider.entity.name == SERVICE_PROVIDER_NAME:
        wos_client.service_providers.delete(service_provider_id=provider.metadata.id)
        break

In [17]:
# Add Service Provider

added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        credentials={},
        operational_space_id="production",
        background_mode=False
    ).result

service_provider_id = added_service_provider_result.metadata.id

wos_client.service_providers.show()




 Waiting for end of adding service provider a700b804-2292-4fcd-93d7-e28a35ba7f6f 






active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




None,active,Final-Drift-V2-Batch - Hive-DO-NOT-DELETE,custom_machine_learning,2025-02-13 09:50:52.697000+00:00,a700b804-2292-4fcd-93d7-e28a35ba7f6f
None,active,CUSTOM_BATCH,custom_machine_learning,2025-02-13 08:23:50.242000+00:00,ced3ab4b-5a9d-41b1-9dda-350e3f38dfac
None,active,GCR batch explain,custom_machine_learning,2025-02-13 08:08:17.832000+00:00,24b159a2-0d0a-4e2a-ac84-656a2d5ca327
None,active,Drift-V2-Batch-,custom_machine_learning,2025-02-12 18:58:30.923000+00:00,b6ac70cd-3666-4b7e-9e10-9d4a35b67a10
None,active,DriftV2-Batch-,custom_machine_learning,2025-02-12 18:54:02.327000+00:00,f939057e-305c-4e36-b300-215a0f855daa
None,active,JDBC-DriftV2,custom_machine_learning,2025-02-12 17:55:41.394000+00:00,bfd30f97-cd3b-41e2-ab4a-c1562d7f5e80
99999999-9999-9999-9999-999999999999,active,SPACE_xgov_prod,watson_machine_learning,2025-02-11 06:06:41.995000+00:00,8c640071-25e1-4be3-87c5-bf19b7dc9715
99999999-9999-9999-9999-999999999999,active,SPACE_xgov_pre_prod,watson_machine_learning,2025-02-11 05:57:21.485000+00:00,cfa4d4f3-2367-4b7f-a418-9a71f0bb8b1c
99999999-9999-9999-9999-999999999999,active,PROJECT_Rag demos,watson_machine_learning,2025-02-11 05:30:35.020000+00:00,ede39287-d476-4fc4-a58d-5c416f66c865
None,active,DELETE-BATCH-TEST,custom_machine_learning,2025-02-06 06:53:38.394000+00:00,1796a607-8dcd-421d-9fda-ad25c8caf64e


Note: First 10 records were displayed.


In [ ]:
service_provide_details = wos_client.service_providers.get(service_provider_id=service_provider_id).result
print(service_provide_details)

### Create integrated systems for Spark Engine and Hive

In [19]:
# Delete existing spark and hive integrated systems if present

integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems

for system in integrated_systems:
    if system.entity.name in (IAE_SPARK_NAME, HIVE_CONNECTION_NAME):
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

#### Spark Engine

In [ ]:
spark_engine_details = IntegratedSystems(wos_client).add(
    name=IAE_SPARK_NAME,
    description=IAE_SPARK_DESCRIPTION,
    type="spark",
    credentials={
        "username": IBM_CPD_USERNAME,
        "apikey": IBM_CPD_APIKEY
    },
    connection={
        "display_name": IAE_SPARK_DISPLAY_NAME,
        "endpoint": IAE_SPARK_JOBS_ENDPOINT,
        "volume": IBM_CPD_VOLUME,
        "location_type": "cpd_iae"
    }
).result

spark_engine_id = spark_engine_details.metadata.id
print(spark_engine_details)

#### Hive

In [ ]:
hive_connection = {}
hive_credentials = {}
if HIVE_METASTORE_URI is not None:
    hive_connection["metastore_url"] = HIVE_METASTORE_URI
    hive_connection["location_type"] = "metastore"

if KERBEROS_ENABLED is True:
    hive_connection["kerberos_enabled"] = True
    hive_credentials["kerberos_principal"] = HIVE_KERBEROS_PRINCIPAL
    if DELEGATION_TOKEN_SECRET_URN:
        hive_credentials["delegation_token_urn"] = DELEGATION_TOKEN_SECRET_URN
    if DELEGATION_TOKEN_ENDPOINT:
        hive_credentials["delegation_token_endpoint"] = DELEGATION_TOKEN_ENDPOINT

hive_connection_details = IntegratedSystems(wos_client).add(
    name=HIVE_CONNECTION_NAME,
    description=HIVE_CONNECTION_DESCRIPTION,
    type="hive",
    credentials=hive_credentials,
    connection=hive_connection
).result

hive_connection_id=hive_connection_details.metadata.id
print(hive_connection_details)

# 3. Set up a subscription<a name="subscription"></a>

In [ ]:
# Delete an existing subscription with the provided name

subscriptions = wos_client.subscriptions.list().result.subscriptions
for sub in subscriptions:
    if sub.entity.deployment.name == SUBSCRIPTION_NAME:
        wos_client.subscriptions.delete(subscription_id=sub.metadata.id)
        break

# Display all subscriptions
wos_client.subscriptions.show()

### Set subscription metadata

In the following cell, type a path to the common configuration JSON file that you created by running the [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb). After you edit the path information, run the cell to set the asset details and properties, the deployment details, the analytics engine details, and to add the required tables as data sources.

In [22]:
import uuid

common_configuration = None
with open("/path/to/dir/containing/common_config.json", "r") as fp:
    configuration_json = json.load(fp)
    common_configuration = configuration_json.get("common_configuration")
    if common_configuration is None:
        print("Please provide the correct path to the common configuration JSON")
    
# Set asset details
asset = Asset(
    asset_id=str(uuid.uuid4()),
    url="",
    name=SUBSCRIPTION_NAME,
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=ProblemType.BINARY_CLASSIFICATION
)

# Set deployment details
asset_deployment = AssetDeploymentRequest(
    deployment_id=str(uuid.uuid4()),
    name=SUBSCRIPTION_NAME,
    description=SUBSCRIPTION_DESCRIPTION,
    deployment_type="batch"
)

# Set asset properties 
asset_properties_request = AssetPropertiesRequest(
    label_column="Risk",
    probability_fields=[common_configuration["probability"]],
    prediction_field=common_configuration["prediction"],
    feature_fields=common_configuration["feature_columns"],
    categorical_fields=common_configuration["categorical_columns"],
    training_data_schema=SparkStruct.from_dict(common_configuration["training_data_schema"]),
    input_data_schema=SparkStruct.from_dict(common_configuration["input_data_schema"]),
    output_data_schema=SparkStruct.from_dict(common_configuration["output_data_schema"])
)

# Set analytics engine details
analytics_engine = AnalyticsEngine(
    type="spark",
    integrated_system_id=spark_engine_id,
    parameters = spark_parameters
)

# Add selected tables as data sources
data_sources = []
if FEEDBACK_DATABASE_NAME is not None and FEEDBACK_TABLE_NAME is not None:
    feedback_data_source = DataSource(
        type="feedback", 
        database_name=FEEDBACK_DATABASE_NAME,
        schema_name=FEEDBACK_DATABASE_NAME,
        table_name=FEEDBACK_TABLE_NAME, 
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_connection_id
        ),
        parameters={
            # "hive_storage_format": "csv" #supported values are "csv", "parquet", "orc"
        },
        auto_create=False, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    data_sources.append(feedback_data_source)
    
if PAYLOAD_DATABASE_NAME is not None and PAYLOAD_TABLE_NAME is not None \
    and DRIFT_DATABASE_NAME is not None and DRIFT_TABLE_NAME is not None:
    payload_logging_data_source = DataSource(
        type="payload", 
        database_name=PAYLOAD_DATABASE_NAME,
        schema_name=PAYLOAD_DATABASE_NAME,
        table_name=PAYLOAD_TABLE_NAME, 
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_connection_id
        ),
        parameters={
            # "hive_storage_format": "csv" #supported values are "csv", "parquet", "orc"
        },
        auto_create=False, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    drifted_transactions_table_data_source = DataSource(
        type="drift", 
        database_name=DRIFT_DATABASE_NAME,
        schema_name=DRIFT_DATABASE_NAME,
        table_name=DRIFT_TABLE_NAME, 
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_connection_id
        ),
        auto_create=False, #set it to False if table already exists
        status=DataSourceStatus(state="new"),
        # parameters={"hive_storage_format": "csv"}
    )
    
    data_sources.append(payload_logging_data_source)
    data_sources.append(drifted_transactions_table_data_source)

if EXPLAINABILITY_DATABASE_NAME is not None and \
    EXPLAINABILITY_QUEUE_TABLE_NAME is not None and \
        EXPLAINABILITY_RESULT_TABLE_NAME is not None:

    explainability_queue_data_source = DataSource(
        type="explain_queue", 
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_DATABASE_NAME,
        table_name=EXPLAINABILITY_QUEUE_TABLE_NAME,
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_connection_id
        ),
        parameters={
            # "hive_storage_format": "csv" #supported values are "csv", "parquet", "orc"
        },
        auto_create=False, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_queue_data_source)

    explainability_result_data_source = DataSource(
        type="explain_result", 
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_DATABASE_NAME,
        table_name=EXPLAINABILITY_RESULT_TABLE_NAME,
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_connection_id
        ),
        parameters={"hive_storage_format": "csv"},
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_result_data_source)

print(data_sources)

[<ibm_watson_openscale.base_classes.watson_open_scale_v2.DataSource object at 0x7f9f4f698f10>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.DataSource object at 0x7f9f4f699010>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.DataSource object at 0x7f9f4f699090>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.DataSource object at 0x7f9f4f699210>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.DataSource object at 0x7f9f4f699190>]


In [23]:
# Adding the subscription

subscription_details = Subscriptions(wos_client).add(
    data_mart_id=data_mart_id,
    service_provider_id=service_provider_id,
    asset=asset,
    deployment=asset_deployment,
    asset_properties=asset_properties_request,
    analytics_engine=analytics_engine,
    data_sources=data_sources).result

subscription_id = subscription_details.metadata.id
print(subscription_details)

{
  "metadata": {
    "id": "1af22c8d-1d9c-4ccc-b764-ce57bf74856c",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:1af22c8d-1d9c-4ccc-b764-ce57bf74856c",
    "url": "/v2/subscriptions/1af22c8d-1d9c-4ccc-b764-ce57bf74856c",
    "created_at": "2025-02-13T09:50:59.216000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "service_provider_id": "a700b804-2292-4fcd-93d7-e28a35ba7f6f",
    "asset": {
      "asset_id": "5e26abd1-4611-46e9-9f34-6e9cb3ab537b",
      "url": "",
      "name": "Final-Drift-V2-Batch - Hive-DO-NOT-DELETE",
      "asset_type": "model",
      "problem_type": "binary",
      "input_data_type": "structured"
    },
    "asset_properties": {
      "training_data_schema": {
        "type": "struct",
        "fields": [
          {
            "metadata": {
              "columnInfo": {
                "columnLength": 64
              },
            

In [27]:
# Check subscription status
import time

subscription_status = None
while subscription_status not in ("active", "error"):
    subscription_status = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    if subscription_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), subscription_status)
        time.sleep(15)
        
print(datetime.now().strftime("%H:%M:%S"), subscription_status)

# print(wos_client.subscriptions.get(subscription_id).result.entity)

09:51:05 preparing


09:51:21 preparing


09:51:36 preparing


09:51:51 preparing


09:52:06 active


# 4. Quality monitoring <a name="quality"></a>

### Enable the quality monitor

In the following code cell, default values are set for the quality monitor. You can change the default values by updating the optional `min_feedback_data_size` attribute in the `parameters` dict and set the quality threshold in the `thresholds` list.

In [28]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_feedback_data_size": 10
}

thresholds = [{
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
}]

quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

quality_monitor_instance_id = quality_monitor_details.metadata.id
print(quality_monitor_details)

{
  "metadata": {
    "id": "02a7f40b-f784-4e2b-aed5-9c58abc1718d",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:02a7f40b-f784-4e2b-aed5-9c58abc1718d",
    "url": "/v2/monitor_instances/02a7f40b-f784-4e2b-aed5-9c58abc1718d",
    "created_at": "2025-02-13T09:52:10.798000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "1af22c8d-1d9c-4ccc-b764-ce57bf74856c"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "type": "relative",
        "delay_unit": "minute",
        "delay": 10
      },
      "rep

### Check monitor instance status

In [29]:
monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "02a7f40b-f784-4e2b-aed5-9c58abc1718d",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:02a7f40b-f784-4e2b-aed5-9c58abc1718d",
    "url": "/v2/monitor_instances/02a7f40b-f784-4e2b-aed5-9c58abc1718d",
    "created_at": "2025-02-13T09:52:10.798000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "1af22c8d-1d9c-4ccc-b764-ce57bf74856c"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "type": "relative",
        "delay_unit": "minute",
        "delay": 10
      },
      "rep

In [30]:
from datetime import datetime

quality_status = None

while quality_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
    quality_status = monitor_instance_details.entity.status.state
    if quality_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), quality_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), quality_status)

09:52:11 preparing


09:52:41 active


### Run an on-demand evaluation

In [31]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=
                                                          quality_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "6b5c6d26-301d-4b05-a541-52626d9dadc7",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:6b5c6d26-301d-4b05-a541-52626d9dadc7",
    "url": "/v2/monitor_instances/02a7f40b-f784-4e2b-aed5-9c58abc1718d/runs/6b5c6d26-301d-4b05-a541-52626d9dadc7",
    "created_at": "2025-02-13T09:52:41.507000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "min_feedback_data_size": 10
    },
    "status": {
      "state": "running",
      "queued_at": "2025-02-13T09:52:41.501000Z",
      "started_at": "2025-02-13T09:52:41.507000Z",
      "operators": []
    }
  }
}


In [32]:
# Check run status
import time
quality_run_status = None
while quality_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=quality_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    quality_run_status = monitoring_run_details.entity.status.state
    if quality_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), quality_run_status)
        time.sleep(30)
print(monitoring_run_details.entity)

09:52:41 running


09:53:11 running


09:53:41 running


09:54:11 running


09:54:41 running


...

09:56:12 running


09:56:42 running


09:57:12 running


09:57:42 running


{
  "triggered_by": "user",
  "parameters": {
    "job_run_schedule_id": "a4746cba-3ca4-4dcb-ab04-58b3b8e44c1f",
    "min_feedback_data_size": 10,
    "total_records_processed": 1000
  },
  "status": {
    "state": "finished",
    "queued_at": "2025-02-13T09:52:41.501000Z",
    "started_at": "2025-02-13T09:52:42.064000Z",
    "updated_at": "2025-02-13T09:57:43.472000Z",
    "completed_at": "2025-02-13T09:57:43.416000Z",
    "message": "Quality execution is completed.",
    "operators": [
      {
        "id": "original",
        "status": {
          "state": "finished",
          "completed_at": "2025-02-13T09:57:43.416000Z"
        }
      }
    ]
  }
}


### Display quality metrics

In [33]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2025-02-13 09:57:43.218071+00:00,true_positive_rate,c61ea681-c9a2-46aa-9302-feeafeeaca37,0.09968847352024922,None,None,['model_type:original'],quality,02a7f40b-f784-4e2b-aed5-9c58abc1718d,6b5c6d26-301d-4b05-a541-52626d9dadc7,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 09:57:43.218071+00:00,area_under_roc,c61ea681-c9a2-46aa-9302-feeafeeaca37,0.4975614679825105,0.8,None,['model_type:original'],quality,02a7f40b-f784-4e2b-aed5-9c58abc1718d,6b5c6d26-301d-4b05-a541-52626d9dadc7,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 09:57:43.218071+00:00,precision,c61ea681-c9a2-46aa-9302-feeafeeaca37,0.3106796116504854,None,None,['model_type:original'],quality,02a7f40b-f784-4e2b-aed5-9c58abc1718d,6b5c6d26-301d-4b05-a541-52626d9dadc7,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 09:57:43.218071+00:00,f1_measure,c61ea681-c9a2-46aa-9302-feeafeeaca37,0.1509433962264151,None,None,['model_type:original'],quality,02a7f40b-f784-4e2b-aed5-9c58abc1718d,6b5c6d26-301d-4b05-a541-52626d9dadc7,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 09:57:43.218071+00:00,accuracy,c61ea681-c9a2-46aa-9302-feeafeeaca37,0.64,None,None,['model_type:original'],quality,02a7f40b-f784-4e2b-aed5-9c58abc1718d,6b5c6d26-301d-4b05-a541-52626d9dadc7,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 09:57:43.218071+00:00,log_loss,c61ea681-c9a2-46aa-9302-feeafeeaca37,0.6580384153493594,None,None,['model_type:original'],quality,02a7f40b-f784-4e2b-aed5-9c58abc1718d,6b5c6d26-301d-4b05-a541-52626d9dadc7,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 09:57:43.218071+00:00,false_positive_rate,c61ea681-c9a2-46aa-9302-feeafeeaca37,0.10456553755522828,None,None,['model_type:original'],quality,02a7f40b-f784-4e2b-aed5-9c58abc1718d,6b5c6d26-301d-4b05-a541-52626d9dadc7,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 09:57:43.218071+00:00,area_under_pr,c61ea681-c9a2-46aa-9302-feeafeeaca37,0.3153253939448931,None,None,['model_type:original'],quality,02a7f40b-f784-4e2b-aed5-9c58abc1718d,6b5c6d26-301d-4b05-a541-52626d9dadc7,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 09:57:43.218071+00:00,recall,c61ea681-c9a2-46aa-9302-feeafeeaca37,0.09968847352024922,None,None,['model_type:original'],quality,02a7f40b-f784-4e2b-aed5-9c58abc1718d,6b5c6d26-301d-4b05-a541-52626d9dadc7,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c


# 5. Drift monitoring <a name="drift"></a>

### Enable the drift monitor

In the following code cell, type a path to the drift configuration tar ball.

In [35]:
wos_client.monitor_instances.upload_drift_model(
    model_path="/path/to/dir/containing/drift_archive.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

{'data_constraints': {'id': 'eb7b77e5-29ab-4ac9-a60e-92a0fbfc2acc',
  'version': '0.02_batch',
  'columns': [{'name': 'Age',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CheckingStatus',
    'dtype': 'categorical',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CreditHistory',
    'dtype': 'categorical',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CurrentResidenceDuration',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'Dependents',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'EmploymentDuration',
    'dtype': 'categorical',
    'count': 1000,
    'appr

In the following code cell, default values are set for the drift monitor. You can change the default values by updating the values in the `parameters` section. The `min_samples` parameter controls the number of records that triggers the drift monitor to run. The `drift_threshold` parameter sets the threshold in decimal format for the drift percentage to trigger an alert. The `train_drift_model` parameter controls whether to re-train the model based on the drift analysis.

In [36]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": False
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
print(drift_monitor_details)

{
  "metadata": {
    "id": "9c7f6eee-6cad-4c42-969f-3ab174ff230d",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:9c7f6eee-6cad-4c42-969f-3ab174ff230d",
    "url": "/v2/monitor_instances/9c7f6eee-6cad-4c42-969f-3ab174ff230d",
    "created_at": "2025-02-13T09:58:13.915000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "1af22c8d-1d9c-4ccc-b764-ce57bf74856c"
    },
    "parameters": {
      "drift_threshold": 0.05,
      "min_samples": 100,
      "train_drift_model": false
    },
    "thresholds": [
      {
        "metric_id": "drift_magnitude",
        "type": "upper_limit",
        "value": 0.5
      },
      {
        "metric_id": "predicted_accuracy",
        "type": "upper_limit",
        "value": 0.8
      },
      {
        "metric_id": "data_d

### Check monitor instance status

In [37]:
drift_status = None

while drift_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
    drift_status = monitor_instance_details.entity.status.state
    if drift_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)

09:58:14 preparing


09:58:44 preparing


09:59:14 preparing


09:59:44 preparing


10:00:14 preparing


10:00:44 active


### Run an on-demand evaluation

In [38]:
# Check Drift monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "9c7f6eee-6cad-4c42-969f-3ab174ff230d",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:9c7f6eee-6cad-4c42-969f-3ab174ff230d",
    "url": "/v2/monitor_instances/9c7f6eee-6cad-4c42-969f-3ab174ff230d",
    "created_at": "2025-02-13T09:58:13.915000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-02-13T10:00:17.835000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "1af22c8d-1d9c-4ccc-b764-ce57bf74856c"
    },
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": null,
      "drift_threshold": 0

In [39]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "dd203f7c-e629-4f02-afdc-d204db1167b0",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:dd203f7c-e629-4f02-afdc-d204db1167b0",
    "url": "/v2/monitor_instances/9c7f6eee-6cad-4c42-969f-3ab174ff230d/runs/dd203f7c-e629-4f02-afdc-d204db1167b0",
    "created_at": "2025-02-13T10:00:44.535000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": null,
      "drift_threshold": 0.05,
      "min_samples": 100,
      "model_drift_enabled": true,
      "table_schema": {
        "fields": [
          {
            "length": 64,
            "metadata": {},
            "name": "scoring_id",
            "nullable": false,
       

In [40]:
# Check run status

drift_run_status = None
while drift_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_run_status = monitoring_run_details.entity.status.state
    if drift_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), drift_run_status)
        time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), drift_run_status)


10:00:44 running


10:01:14 running


10:01:44 running


10:02:14 running


10:02:45 running


10:03:15 finished


### Display drift metrics

In [41]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2025-02-13 10:00:46.221354+00:00,data_drift_magnitude,efb5e350-6af1-4f58-8b49-415c8923d15a,0.004807692307692308,None,0.1,[],drift,9c7f6eee-6cad-4c42-969f-3ab174ff230d,dd203f7c-e629-4f02-afdc-d204db1167b0,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:00:46.221354+00:00,drift_magnitude,efb5e350-6af1-4f58-8b49-415c8923d15a,0.0,None,0.05,[],drift,9c7f6eee-6cad-4c42-969f-3ab174ff230d,dd203f7c-e629-4f02-afdc-d204db1167b0,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:00:46.221354+00:00,predicted_accuracy,efb5e350-6af1-4f58-8b49-415c8923d15a,0.8076923076923077,None,None,[],drift,9c7f6eee-6cad-4c42-969f-3ab174ff230d,dd203f7c-e629-4f02-afdc-d204db1167b0,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c


# 6. Drift V2 monitoring <a name="drift_v2"></a>

### Enable the drift v2 monitor

In the following code cell, type a path to the drift v2 configuration tar ball.

In [42]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift_v2" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift_v2 monitor instance with id: ', monitor_instance.metadata.id)

In [44]:
# Upload drift_v2 archive
wos_client.monitor_instances.upload_drift_v2_archive(
    archive_path="/path/to/dir/containing/drift_v2_archive.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

{}

In [45]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
        "min_samples": 10,
        "max_samples": 1000,
        "train_archive": False
    }
drift_v2_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT_V2.ID,
    target=target,
    parameters=parameters
).result

drift_v2_monitor_instance_id = drift_v2_monitor_details.metadata.id
drift_v2_monitor_instance_id

'ca7b3970-4322-4433-9594-bd14d3ec951f'

### Check monitor instance status

In [47]:
import time 
drift_v2_status = None

while drift_v2_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_v2_monitor_instance_id).result
    drift_v2_status = monitor_instance_details.entity.status.state
    if drift_v2_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_v2_status)
        time.sleep(30)

print(datetime.now().strftime("%H:%M:%S"), drift_v2_status)

10:03:17 preparing


10:03:47 active


### Trigger on-demand monitor run

In [48]:
# Check Drift v2 monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_v2_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "ca7b3970-4322-4433-9594-bd14d3ec951f",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:ca7b3970-4322-4433-9594-bd14d3ec951f",
    "url": "/v2/monitor_instances/ca7b3970-4322-4433-9594-bd14d3ec951f",
    "created_at": "2025-02-13T10:03:17.327000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-02-13T10:03:35.444000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift_v2",
    "target": {
      "target_type": "subscription",
      "target_id": "1af22c8d-1d9c-4ccc-b764-ce57bf74856c"
    },
    "parameters": {
      "advanced_controls": {
        "enable_drift_v2_batch": true
      },
      "context": {
        "baseline": {
          "data_set_id": "e32f39c1-e782-4a55-89c6-66eb4e7bb3b1"
        },
        "copy_subscription": {
          "enabled": false,
          "source_id": null
   

In [49]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_v2_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "5bda1dc1-b3a3-42bb-9ca4-9af0bb077575",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:5bda1dc1-b3a3-42bb-9ca4-9af0bb077575",
    "url": "/v2/monitor_instances/ca7b3970-4322-4433-9594-bd14d3ec951f/runs/5bda1dc1-b3a3-42bb-9ca4-9af0bb077575",
    "created_at": "2025-02-13T10:03:48.036000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "advanced_controls": {
        "enable_drift_v2_batch": true
      },
      "context": {
        "baseline": {
          "data_set_id": "e32f39c1-e782-4a55-89c6-66eb4e7bb3b1"
        },
        "copy_subscription": {
          "enabled": false,
          "source_id": null
        },
        "data_sets": {
          "drift_insights": "691f53a6-34f7-453c-a562-73680e6e4c6d",
          "drift_intervals": "25cdb5cf-34f8-4f85-9f22-7e232d86bf3d",
          "drift_scores": "2602e16a-def8-48c1-bbdc-916d8f60ac79",
          "drift_

In [50]:
# Check run status
drift_v2_run_status = None
while drift_v2_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_v2_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_v2_run_status = monitoring_run_details.entity.status.state
    if drift_v2_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_v2_run_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), drift_v2_run_status)

10:03:48 running


10:04:18 running


10:04:48 running


10:05:18 running


10:05:48 running


...

10:13:19 running


10:13:49 running


10:14:19 running


10:14:49 running


10:15:19 finished


### Check Drift V2 metrics

In [51]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_v2_monitor_instance_id)

2025-02-13 10:11:02.721658+00:00,records_processed,cc14a7a1-66ea-4fd3-8cb1-2d0a6034bda9,208.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,ca7b3970-4322-4433-9594-bd14d3ec951f,5bda1dc1-b3a3-42bb-9ca4-9af0bb077575,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:11:02.721658+00:00,confidence_drift_score,cc14a7a1-66ea-4fd3-8cb1-2d0a6034bda9,0.4599,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,ca7b3970-4322-4433-9594-bd14d3ec951f,5bda1dc1-b3a3-42bb-9ca4-9af0bb077575,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:11:02.721658+00:00,records_processed,cc14a7a1-66ea-4fd3-8cb1-2d0a6034bda9,208.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,ca7b3970-4322-4433-9594-bd14d3ec951f,5bda1dc1-b3a3-42bb-9ca4-9af0bb077575,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:11:02.721658+00:00,confidence_drift_score,cc14a7a1-66ea-4fd3-8cb1-2d0a6034bda9,0.4605,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,ca7b3970-4322-4433-9594-bd14d3ec951f,5bda1dc1-b3a3-42bb-9ca4-9af0bb077575,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:11:11.821410+00:00,records_processed,d9da7936-4a7f-4f5a-8981-45cb41727a62,208.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,ca7b3970-4322-4433-9594-bd14d3ec951f,5bda1dc1-b3a3-42bb-9ca4-9af0bb077575,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:11:11.821410+00:00,confidence_drift_score,d9da7936-4a7f-4f5a-8981-45cb41727a62,0.4589,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,ca7b3970-4322-4433-9594-bd14d3ec951f,5bda1dc1-b3a3-42bb-9ca4-9af0bb077575,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:11:11.821410+00:00,records_processed,d9da7936-4a7f-4f5a-8981-45cb41727a62,208.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,ca7b3970-4322-4433-9594-bd14d3ec951f,5bda1dc1-b3a3-42bb-9ca4-9af0bb077575,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:11:11.821410+00:00,confidence_drift_score,d9da7936-4a7f-4f5a-8981-45cb41727a62,0.4573,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,ca7b3970-4322-4433-9594-bd14d3ec951f,5bda1dc1-b3a3-42bb-9ca4-9af0bb077575,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:11:18.122157+00:00,records_processed,b8dfdaf4-e157-4384-a731-25d22aa54e4e,208.0,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:prediction']",drift_v2,ca7b3970-4322-4433-9594-bd14d3ec951f,5bda1dc1-b3a3-42bb-9ca4-9af0bb077575,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:11:18.122157+00:00,prediction_drift_score,b8dfdaf4-e157-4384-a731-25d22aa54e4e,0.0062,None,0.05,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:prediction']",drift_v2,ca7b3970-4322-4433-9594-bd14d3ec951f,5bda1dc1-b3a3-42bb-9ca4-9af0bb077575,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c


Note: First 10 records were displayed.


# 7. Fairness monitoring <a name="drift_v2"></a>

### Enable the fairness monitor

The following code cell, will enable the fairness monitor.

In [52]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = configuration_json["fairness_configuration"]["parameters"]
thresholds = configuration_json["fairness_configuration"]["thresholds"]

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

fairness_monitor_instance_id = fairness_monitor_details.metadata.id
print(fairness_monitor_details)

{
  "metadata": {
    "id": "27c42d73-dec7-41c9-85a1-f9bf79b28d44",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:27c42d73-dec7-41c9-85a1-f9bf79b28d44",
    "url": "/v2/monitor_instances/27c42d73-dec7-41c9-85a1-f9bf79b28d44",
    "created_at": "2025-02-13T10:15:21.710000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "fairness",
    "target": {
      "target_type": "subscription",
      "target_id": "1af22c8d-1d9c-4ccc-b764-ce57bf74856c"
    },
    "parameters": {
      "training_data_last_processed_time": "2025-01-17T09:40:55.219953Z",
      "min_records": 100,
      "features": [
        {
          "minority": [
            "female"
          ],
          "feature": "Sex",
          "metric_ids": [
            "fairness_value",
            "statistical_parity_difference",
            "average_odds_difference",
            "av

### Check monitor instance status

In [53]:
fairness_state = fairness_monitor_details.entity.status.state

while fairness_state not in ("active", "error"):
    print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=fairness_monitor_instance_id).result
    fairness_state = monitor_instance_details.entity.status.state
    time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)

10:15:21 preparing


10:15:51 preparing


10:16:22 active


### Run an on-demand evaluation

In [54]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "8533765e-72de-4a44-9b53-94dd44b06953",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:8533765e-72de-4a44-9b53-94dd44b06953",
    "url": "/v2/monitor_instances/27c42d73-dec7-41c9-85a1-f9bf79b28d44/runs/8533765e-72de-4a44-9b53-94dd44b06953",
    "created_at": "2025-02-13T10:16:22.281000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "favourable_class": [
        "No Risk"
      ],
      "features": [
        {
          "feature": "Sex",
          "majority": [
            "male"
          ],
          "metric_ids": [
            "fairness_value",
            "statistical_parity_difference",
            "average_odds_difference",
            "average_abs_odds_difference",
            "false_negative_rate_difference",
            "false_positive_rate_difference",
            "false_discovery_rate_difference",
            "false_omission_rate_differenc

In [55]:
# Check run status

fairness_run_status = monitoring_run_details.entity.status.state
while fairness_run_status not in ("finished", "error"):
    print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=fairness_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    fairness_run_status = monitoring_run_details.entity.status.state
    time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)

10:16:22 running


10:16:52 running


10:17:22 running


10:17:52 running


10:18:23 running


...

10:20:23 running


10:20:53 running


10:21:23 running


10:21:54 running


10:22:24 finished


### Display fairness metrics

In [56]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2025-02-13 10:21:26.403330+00:00,false_positive_rate_difference,0c51959b-ebfb-48b7-8961-128a20843380,-0.141,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,27c42d73-dec7-41c9-85a1-f9bf79b28d44,8533765e-72de-4a44-9b53-94dd44b06953,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:21:26.403330+00:00,false_negative_rate_difference,0c51959b-ebfb-48b7-8961-128a20843380,0.115,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,27c42d73-dec7-41c9-85a1-f9bf79b28d44,8533765e-72de-4a44-9b53-94dd44b06953,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:21:26.403330+00:00,error_rate_difference,0c51959b-ebfb-48b7-8961-128a20843380,0.0,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,27c42d73-dec7-41c9-85a1-f9bf79b28d44,8533765e-72de-4a44-9b53-94dd44b06953,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:21:26.403330+00:00,false_omission_rate_difference,0c51959b-ebfb-48b7-8961-128a20843380,-0.007,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,27c42d73-dec7-41c9-85a1-f9bf79b28d44,8533765e-72de-4a44-9b53-94dd44b06953,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:21:26.403330+00:00,fairness_value,0c51959b-ebfb-48b7-8961-128a20843380,103.9911308203991,95.0,125.0,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,27c42d73-dec7-41c9-85a1-f9bf79b28d44,8533765e-72de-4a44-9b53-94dd44b06953,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:21:26.403330+00:00,statistical_parity_difference,0c51959b-ebfb-48b7-8961-128a20843380,0.031,-0.3,0.3,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,27c42d73-dec7-41c9-85a1-f9bf79b28d44,8533765e-72de-4a44-9b53-94dd44b06953,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:21:26.403330+00:00,false_discovery_rate_difference,0c51959b-ebfb-48b7-8961-128a20843380,-0.051,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,27c42d73-dec7-41c9-85a1-f9bf79b28d44,8533765e-72de-4a44-9b53-94dd44b06953,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:21:26.403330+00:00,average_odds_difference,0c51959b-ebfb-48b7-8961-128a20843380,-0.128,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,27c42d73-dec7-41c9-85a1-f9bf79b28d44,8533765e-72de-4a44-9b53-94dd44b06953,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:21:26.403330+00:00,average_abs_odds_difference,0c51959b-ebfb-48b7-8961-128a20843380,0.128,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,27c42d73-dec7-41c9-85a1-f9bf79b28d44,8533765e-72de-4a44-9b53-94dd44b06953,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c
2025-02-13 10:21:26.403330+00:00,false_positive_rate_difference,0c51959b-ebfb-48b7-8961-128a20843380,0.006,-0.15,0.15,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,27c42d73-dec7-41c9-85a1-f9bf79b28d44,8533765e-72de-4a44-9b53-94dd44b06953,subscription,1af22c8d-1d9c-4ccc-b764-ce57bf74856c


Note: First 10 records were displayed.


# 8. Explainability monitoring <a name="explainability"></a>

### Enable the explainability monitor

#### Upload explainability configuration archive
In the following code cell, type the path to the explainability configuration archive tar ball.

In [66]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "explainability" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing explainability monitor instance with id: ', monitor_instance.metadata.id)

Deleted existing explainability monitor instance with id:  f23068ee-2840-4116-a9b1-bde7231e78ed


In [68]:
with open("explainability.tar.gz", mode="rb") as explainability_tar:
    wos_client.monitor_instances.upload_explainability_archive(subscription_id=subscription_id, archive=explainability_tar)

print("Uploaded explainability archive successfully.")

Uploaded explainability archive successfully.


In [69]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
# Uncomment the below lines to enable lime global explanation. Available from Cloud Pak for Data version 4.6.4 onwards.
#    "global_explanation": {
#        "enabled": True,  # Flag to enable global explanation 
#        "explanation_method": "lime",
#        "sample_size": 1000, # [Optional] The sample size of records to be used for generating payload data global explanation. If not specified entire data in the payload window is used.
#    }
}

explainability_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_instance_id = explainability_monitor_details.metadata.id
print(explainability_monitor_details)

{
  "metadata": {
    "id": "f8122615-2589-4589-8966-5b88e13ae550",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:f8122615-2589-4589-8966-5b88e13ae550",
    "url": "/v2/monitor_instances/f8122615-2589-4589-8966-5b88e13ae550",
    "created_at": "2025-02-13T10:28:08.944000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "1af22c8d-1d9c-4ccc-b764-ce57bf74856c"
    },
    "parameters": {},
    "thresholds": [
      {
        "metric_id": "global_explanation_stability",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "type": "relative",
        "delay_unit": "minute",
        "delay": 10
      },
      "repeat_type": "week"
 

### Check monitor instance status

In [70]:
explainability_status = None

while explainability_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
    explainability_status = monitor_instance_details.entity.status.state
    if explainability_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)

10:28:09 preparing


10:28:39 active


### Run an on-demand evaluation

In [71]:
# Check Explainbility monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "f8122615-2589-4589-8966-5b88e13ae550",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:f8122615-2589-4589-8966-5b88e13ae550",
    "url": "/v2/monitor_instances/f8122615-2589-4589-8966-5b88e13ae550",
    "created_at": "2025-02-13T10:28:08.944000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-02-13T10:28:10.665000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "1af22c8d-1d9c-4ccc-b764-ce57bf74856c"
    },
    "parameters": {
      "config_modified_at": "2025-02-13T10:28:10.491799Z",
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "lime": {
        "enabled": true,
        "features_count": 10,
        "perturbations_count": 10000
      }
    },
    "thresholds

In [72]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=explainability_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id
print(monitoring_run_details)

{
  "metadata": {
    "id": "5424ef4c-d9a0-428c-81c9-8a74cf719623",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:5424ef4c-d9a0-428c-81c9-8a74cf719623",
    "url": "/v2/monitor_instances/f8122615-2589-4589-8966-5b88e13ae550/runs/5424ef4c-d9a0-428c-81c9-8a74cf719623",
    "created_at": "2025-02-13T10:30:14.454000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "config_modified_at": "2025-02-13T10:28:10.491799Z",
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "lime": {
        "enabled": true,
        "features_count": 10,
        "perturbations_count": 10000
      }
    },
    "status": {
      "state": "running",
      "queued_at": "2025-02-13T10:30:14.449000Z",
      "started_at": "2025-02-13T10:30:14.454000Z",
      "operators": []
    }
  }
}


In [73]:
# Check run status

explainability_run_status = None
while explainability_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=explainability_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    explainability_run_status = monitoring_run_details.entity.status.state
    if explainability_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)
        time.sleep(60)
    elif explainability_run_status == "error":
        print(monitoring_run_details.entity)
print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)

10:30:15 running


10:31:15 running


10:32:15 running


10:33:15 running


10:34:15 running


10:35:15 running


10:36:15 finished


In [74]:
# View the global explanation stability metric. When lime global explanation is enabled, the monitor run computes global explanation and publishes global_explanation_stability metric.
# wos_client.monitor_instances.show_metrics(monitor_instance_id=explainability_monitor_instance_id)

### Display sample explanations

In [75]:
explanations = wos_client.monitor_instances.get_all_explaination_tasks(subscription_id=subscription_id).result
print(explanations)

{
  "total_count": 10,
  "limit": 50,
  "offset": 0,
  "explanation_fields": [
    "explanation_task_id",
    "scoring_id",
    "created_at",
    "finished_at",
    "status",
    "prediction",
    "subscription_id",
    "deployment_id",
    "asset_name",
    "deployment_name",
    "probability",
    "explanation_type"
  ],
  "explanation_values": [
    [
      "09a3c326-5a5e-4379-a2f6-aa22a7ec1b86",
      "11a4add7-c9de-4900-a5dc-96c5b1990638",
      "2025-02-13T10:31:06.087670Z",
      "2025-02-13T10:31:06.087829Z",
      "finished",
      "No Risk",
      "1af22c8d-1d9c-4ccc-b764-ce57bf74856c",
      "adc34fb2-a17b-4d7c-a5b3-42f427cc7c61",
      "Final-Drift-V2-Batch - Hive-DO-NOT-DELETE",
      "Final-Drift-V2-Batch - Hive-DO-NOT-DELETE",
      0.64,
      "lime"
    ],
    [
      "fbacd58f-e505-48f4-aa87-4f54cfec5e56",
      "9af21997-245a-40dc-8c84-a1edfac717b4",
      "2025-02-13T10:31:06.083608Z",
      "2025-02-13T10:31:06.083773Z",
      "finished",
      "No Risk",
      "1a

## Congratulations!

You have finished the Batch demo for IBM Watson OpenScale by using Apache Spark on Cloud Pak for Data IBM Analytics Engine. You can now view the [Watson OpenScale Dashboard](https://url-to-your-cp4d-cluster/aiopenscale). Click the tile for the **German Credit model** to see the quality, drift and fairness monitors. Click the timeseries graph to get detailed information on transactions during a specific time window.